In [72]:
import spacy
import prodigy
import pandas as pd

In [73]:
import os
os.chdir(r'C:\Users\ludwig\Documents\uni_stuff\capstone')

In [82]:
full_df = pd.read_csv('EU_Register_on_nutrition_and_health_claims.csv')
full_df

,Claim type,"Nutrient substance, food or food category",Claim,Conditions of use of the claim / Restrictions of use / Reasons for non-authorisation,Health relationship,EFSA opinion reference,Commission Regulation,Status,Entry Id
0,Art.13(1),Alpha-linolenic acid (ALA),ALA contributes to the maintenance of normal b...,The claim may be used only for food which is a...,maintenance of normal blood cholesterol concen...,2009;7(9):1252,Commission Regulation (EU) 432/2012 of 16/05/2012,Authorised,"493, 568"
1,NaN,NaN,NaN,NaN,NaN,2011;9(6):2203,NaN,NaN,NaN
2,Art.13(1),Activated charcoal,Activated charcoal contributes to reducing exc...,The claim may be used only for food which cont...,reduction of excessive intestinal gas accumula...,2011;9(4):2049,Commission Regulation (EU) 432/2012 of 16/05/2012,Authorised,1938
3,Art.13(1),Barley grain fibre,Barley grain fibre contributes to an increase ...,The claim may be used only for food which is h...,increase in faecal bulk,2011;9(6):2249,Commission Regulation (EU) 432/2012 of 16/05/2012,Authorised,819
4,Art.13(1),Beta-glucans,Beta-glucans contribute to the maintenance of ...,The claim may be used only for food which cont...,maintenance of normal blood cholesterol concen...,2009;7(9):1254,Commission Regulation (EU) 432/2012 of 16/05/2012,Authorised,"754, 755, 757, 801, 1236, 1299, 1465, 2934"
...,...,...,...,...,...,...,...,...,...
2559,Art.14(1)(b),Mixture of bifidobacteria (Bifidobacterium bif...,Probiotic bifidobacteria lead to a healthy int...,Non-compliance with the Regulation because on ...,NaN,Q-2009-00224,Commission Regulation (EU) No 957/2010 of 22/1...,Non-authorised,NaN
2560,Art.14(1)(b),I omega kids®/Pufan 3 kids® containing docosah...,Provide serenity and room for a beneficial dev...,Non-compliance with the Regulation because on ...,NaN,Q-2008-092 Q-2008-097,Commission Regulation (EC) 983/2009 of 21/10/2009,Non-authorised,NaN
2561,Art.14(1)(b),"Dairy products (milk, cheese and yoghurt)","Three portions of dairy food everyday, as part...",Non-compliance with the Regulation because on ...,NaN,Q-2008-110,Commission Regulation (EC) 983/2009 of 21/10/2009,Non-authorised,NaN
2562,Art.14(1)(b),"regulat®.pro.kid IMMUN, food supplement with f...","regulat®.pro.kid IMMUN supports, stimulates an...",Non-compliance with the Regulation because on ...,NaN,Q-2008-082,Commission Regulation (EC) 983/2009 of 21/10/2009,Non-authorised,NaN


In [83]:
full_df = full_df.rename(columns= {' Status' : 'Status' , 'Nutrient substance, food or food category': 'Food', ' Health relationship': 'Phenotype', ' Claim': 'Claim'})

In [35]:
full_df.dropna(subset=['Status', 'Food', 'Claim', 'Phenotype'], inplace = True)

In [36]:
unauth_df = full_df[full_df.Status == 'Non-authorised']

In [38]:
#since a lot of data here is in bullet points, without mentioning the food entities in the claim text, I write a method here that replaces those bullet points with entitiy mentions. While this can lead to bad grammatical structures, overall it should still improve the model

def format_sent(row):
    claim = str(row['Claim']).lower()
    food = str(row['Food']).lower()
    s = claim.replace('-', food + ' ')
    #row['Claim'] = claim.replace('-', food)
    s = s.replace('*', food + ' ')
    #row['Claim'] = claim.replace('*', food)

    return s


In [39]:
better_sent = unauth_df.apply(format_sent, axis = 1)



In [40]:
#append the formatted claim texts as new column
unauth_df['Claim_format'] = better_sent

C:\Users\ludwig\AppData\Local\Temp\ipykernel_5568\855293300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unauth_df['Claim_format'] = better_sent


In [41]:
#create a new dataframe with just claim texts
claims_col = unauth_df[['Claim_format']]

In [42]:
#change the name of the column
claims_col = claims_col.rename(columns = {'Claim_format': 'Claim'})

In [43]:
claims_col

,Claim
348,"""sugar beet fibre the combination of soluble ..."
349,"""the inclusion of 3 grams oat beta glucan solu..."
350,"""traditionally used for the good circulation o..."
351,"""traditionally used to enhance the renal elimi..."
352,"""traditionally used to facilitate the digestio..."
...,...
2341,•\tfor good intestinal health\r\n•\thelps keep...
2342,•\thelps balancing intestinal activity.\r\n•\t...
2343,•\thelps maintain a healthy balance of benefic...
2344,•\tsupports your natural defences\r\n•\thelps ...


In [44]:
def print_col(row):
  claim = str(row['Claim']).lower()
  print (claim)


In [84]:
# this function returns claim texts in dictionary format so that they may be saved as jsonl files
def restructure_prod(row):
  claim_txt = str(row['Claim']).lower()
  s = re.sub(r'\W+ ', ' ', claim_txt)

  claim_dict_format = {'text': s}
  return claim_dict_format  

In [49]:
#create example

text = ['"- The combination of soluble / insoluble fibre in Sugar beet fibre makes the stomach work properly.\r\n- Dietary fibre helps to maintain bowel/colonic function\r\n- Dietary fibre promotes regularity\r\n- Ensures a healthy digestive system/function"', '"The inclusion of 3 grams oat beta glucan soluble fiber per day from Calorie ControlTrim® Oat Bran, as part of a diet low in saturated fat and a healthy lifestyle, can help maintain healthy blood glucose levels. Oat beta glucan soluble fiber from Calorie ControlTrim® Oat Bran, as part of a diet low in saturated fat and a healthy lifestyle, can help maintain healthy blood glucose levels. Calorie ControlTrim® Oat Bran helps maintain healthy blood glucose levels."', '"Traditionally used for the good circulation of blood in microvessels" / "Traditionally used to decrease the sensations of heavy leggs" / "Used for the good circulation of blood in microvessels" / "Helps to decrease the sensations of heavy leggs"']


test_df = pd.DataFrame(data = text)
test_df.columns = ['Claim']

test_df

,Claim
0,"""- The combination of soluble / insoluble fibr..."
1,"""The inclusion of 3 grams oat beta glucan solu..."
2,"""Traditionally used for the good circulation o..."


In [50]:
test_df.apply(print_col, axis = 1)

"- the combination of soluble / insoluble fibre in sugar beet fibre makes the stomach work properly.
- dietary fibre helps to maintain bowel/colonic function
- dietary fibre promotes regularity
- ensures a healthy digestive system/function"
"the inclusion of 3 grams oat beta glucan soluble fiber per day from calorie controltrim® oat bran, as part of a diet low in saturated fat and a healthy lifestyle, can help maintain healthy blood glucose levels. oat beta glucan soluble fiber from calorie controltrim® oat bran, as part of a diet low in saturated fat and a healthy lifestyle, can help maintain healthy blood glucose levels. calorie controltrim® oat bran helps maintain healthy blood glucose levels."
"traditionally used for the good circulation of blood in microvessels" / "traditionally used to decrease the sensations of heavy leggs" / "used for the good circulation of blood in microvessels" / "helps to decrease the sensations of heavy leggs"


0    None
1    None
2    None
dtype: object

In [51]:
format_test_df = test_df.apply(restructure_prod, axis = 1)

In [52]:
for a in format_test_df:
    print (a)

{'text': ' the combination of soluble insoluble fibre in sugar beet fibre makes the stomach work properly dietary fibre helps to maintain bowel/colonic function dietary fibre promotes regularity ensures a healthy digestive system/function"'}
{'text': '"the inclusion of 3 grams oat beta glucan soluble fiber per day from calorie controltrim oat bran as part of a diet low in saturated fat and a healthy lifestyle can help maintain healthy blood glucose levels oat beta glucan soluble fiber from calorie controltrim oat bran as part of a diet low in saturated fat and a healthy lifestyle can help maintain healthy blood glucose levels calorie controltrim oat bran helps maintain healthy blood glucose levels."'}
{'text': '"traditionally used for the good circulation of blood in microvessels "traditionally used to decrease the sensations of heavy leggs "used for the good circulation of blood in microvessels "helps to decrease the sensations of heavy leggs"'}


In [53]:
# write the test data into a json file
import json
with open('test_file.jsonl','w') as f:
  for i in format_test_df:
    string = json.dumps(i)
    f.write(string+'\n')

In [54]:
list_a = format_test_df.tolist()

In [55]:
test_df2 = pd.DataFrame(data = list_a)
test_df2.columns = ['Claim']

test_df2

,Claim
0,the combination of soluble insoluble fibre in...
1,"""the inclusion of 3 grams oat beta glucan solu..."
2,"""traditionally used for the good circulation o..."


In [57]:
#create array of dictionaries to match jsonl format
format_claims = claims_col.apply(restructure_prod, axis=1)

In [58]:
format_claims

348     {'text': '"sugar beet fibre the combination of...
349     {'text': '"the inclusion of 3 grams oat beta g...
350     {'text': '"traditionally used for the good cir...
351     {'text': '"traditionally used to enhance the r...
352     {'text': '"traditionally used to facilitate th...
                              ...                        
2341    {'text': '•	for good intestinal health
•	help...
2342    {'text': '•	helps balancing intestinal activit...
2343    {'text': '•	helps maintain a healthy balance o...
2344    {'text': '•	supports your natural defences
•	...
2345    {'text': ' for people with a sensitive stomach...
Length: 1875, dtype: object

In [81]:
#write into jsonl file
with open('authorised_annotated.jsonl', 'r') as a:
    for i in a:
        x = json.loads(i)
        print(x)

{'text': 'ala contributes to the maintenance of normal blood cholesterol levels', 'spans': [{'text': 'ala', 'start': 0, 'end': 3, 'label': 'Food'}, {'text': 'normal blood cholesterol', 'start': 38, 'end': 62, 'label': 'Phenotype'}]}
{'text': 'activated charcoal contributes to reducing exessive flatulence after eating', 'spans': [{'text': 'activated charcoal', 'start': 0, 'end': 18, 'label': 'Food'}, {'text': 'exessive flatulence', 'start': 43, 'end': 62, 'label': 'Phenotype'}]}
{'text': 'barley grain fibre contributes to an increase in faecal bulk', 'spans': [{'text': 'barley grain fibre', 'start': 0, 'end': 18, 'label': 'Food'}, {'text': 'increase in faecal bulk', 'start': 37, 'end': 60, 'label': 'Phenotype'}]}
{'text': 'beta-glucans contribute to the maintenance of normal blood cholesterol levels', 'spans': [{'text': 'beta-glucans', 'start': 0, 'end': 12, 'label': 'Food'}, {'text': 'normal blood cholesterol', 'start': 46, 'end': 70, 'label': 'Phenotype'}]}
{'text': 'betaine contribut

In [62]:
format_claims

348     {'text': '"sugar beet fibre the combination of...
349     {'text': '"the inclusion of 3 grams oat beta g...
350     {'text': '"traditionally used for the good cir...
351     {'text': '"traditionally used to enhance the r...
352     {'text': '"traditionally used to facilitate th...
                              ...                        
2341    {'text': '•	for good intestinal health
•	help...
2342    {'text': '•	helps balancing intestinal activit...
2343    {'text': '•	helps maintain a healthy balance o...
2344    {'text': '•	supports your natural defences
•	...
2345    {'text': ' for people with a sensitive stomach...
Length: 1875, dtype: object

In [85]:

with open('unauth_claims.jsonl','w') as f:
  for i in format_claims:
    string = json.dumps(i)
    f.write(string+'\n')

In [71]:
#start annotator in prodigy
!python -m prodigy ner.manual ner_add en_core_web_sm unauth_claims.jsonl --label Food,Phenotype

Using 2 label(s): Food, Phenotype

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!



ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8080): only one usage of each socket address (protocol/network address/port) is normally permitted


In [ ]:
#export annotations from prodigy
!python -m prodigy db-out ner_add annotations

✔ Exported 342 annotations from 'ner_add' in database SQLite
C:\Users\ludwig\Documents\uni_stuff\capstone\annotations\ner_add.jsonl


In [60]:
#combine both datasets

data_comb = []

with open('authorised_annotated.jsonl', 'r') as a:
    for i in a:
        x = json.loads(i)
        data_comb.append(x)
    
with open('ner_add.jsonl', 'r') as b:
    for i in b:
        x = json.loads(i)
        data_comb.append(x)

In [61]:
data_comb

[{'text': 'ala contributes to the maintenance of normal blood cholesterol levels',
  'spans': [{'text': 'ala', 'start': 0, 'end': 3, 'label': 'Food'},
   {'text': 'normal blood cholesterol',
    'start': 38,
    'end': 62,
    'label': 'Phenotype'}]},
 {'text': 'activated charcoal contributes to reducing exessive flatulence after eating',
  'spans': [{'text': 'activated charcoal',
    'start': 0,
    'end': 18,
    'label': 'Food'},
   {'text': 'exessive flatulence',
    'start': 43,
    'end': 62,
    'label': 'Phenotype'}]},
 {'text': 'barley grain fibre contributes to an increase in faecal bulk',
  'spans': [{'text': 'barley grain fibre',
    'start': 0,
    'end': 18,
    'label': 'Food'},
   {'text': 'increase in faecal bulk',
    'start': 37,
    'end': 60,
    'label': 'Phenotype'}]},
 {'text': 'beta-glucans contribute to the maintenance of normal blood cholesterol levels',
  'spans': [{'text': 'beta-glucans', 'start': 0, 'end': 12, 'label': 'Food'},
   {'text': 'normal blood ch

In [63]:
#write into a new jsonl file
with open('combined_annotated.jsonl', 'w') as a:
    for i in data_comb:
        string = json.dumps(i)
        a.write(string+'\n')